## NOTEBOOK DESCRIPTION:

This notebook checks the behaviour of the embedding space: 
- It first checks manually the nearest neighbors of some selected channels
- It also compute the user jumper score which is a first barrier on the evaluation of the embedding

In [1]:
import pickle
import os
import sys
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.manifold import TSNE

scriptpath = "/home/jouven/youtube_projects/"
sys.path.append(os.path.abspath(scriptpath))

from helpers.helpers_channels_more_300 import *
from helpers.helpers_channel_embedding import *

scriptpath = "/home/jouven/youtube_projects"
sys.path.append(os.path.abspath(scriptpath))
from helpers.config_threshold_value import *

### Useful functions

In [2]:
# Selected channels and id-index mapping
dict_channel_ind, dict_ind_channel, channels_id = filtered_channels_index_id_mapping()

In [3]:
PATH = '/dlabdata1/youtube_large/jouven/word2vec_pytorch/run_channels_more_300/channel_sampling_then_permutation/CONTEXT_True_20_SUBSAMPLING_True_0.001_LR_0.018/models/embedding0.csv.gz'

In [4]:
channelcrawler = pd.read_csv("/dlabdata1/youtube_large/df_channels_en.tsv.gz", sep='\t')
# Select the rows being in the select channels
channelcrawler = channelcrawler[channelcrawler['channel'].apply(lambda row: row in channels_id)]

In [5]:
channelcrawler.head()

,category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights
0,Gaming,2010-04-29,UC-lHJZR3Gqxm24_Vd_AJ5Yw,PewDiePie,101000000,3956,3.0,2.087
1,Education,2006-09-01,UCbCmjCuTUZos6Inko4u57UQ,Cocomelon - Nursery ...,60100000,458,7.0,2.087
2,Entertainment,2006-09-20,UCpEhnqL0y41EpW2TvWAHD7Q,SET India,56018869,32661,8.0,2.087
3,Howto & Style,2016-11-15,UC295-Dw_tDNtZXFeAPAW6Aw,5-Minute Crafts,60600000,3591,9.0,2.087
4,Sports,2007-05-11,UCJ5v_MCY6GNUBTO8-D3XoAg,WWE,48400000,43421,11.0,2.087


In [6]:
channelcrawler['category_cc'].unique()

array(['Gaming', 'Education', 'Entertainment', 'Howto & Style', 'Sports',
       'Music', 'Film and Animation', 'Comedy', 'Nonprofits & Activism',
       'People & Blogs', 'News & Politics', 'Science & Technology',
       'Pets & Animals', 'Autos & Vehicles', 'Travel & Events', nan],
      dtype=object)

### Manually check the neirest neighbors of some selected channels

First to check how good is the embedding space, we are going to choose a channel and it's k closest channels in the embedding space. By looking at these channels in the YouTube website, we have a sense of how good/bad is the embedding.

In [7]:
# Looking at the neirest neigbors of `Shaaanxo` being a girl fashion channel
nearest_neighbors_id = get_k_nearest_neighbors(PATH, dict_channel_ind['UCMpOz2KEfkSdd5JeIJh_fxw'], dict_ind_channel, k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel']).merge(channelcrawler)
nearest_neighbors

,channel,category_cc,join_date,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights
0,UCMpOz2KEfkSdd5JeIJh_fxw,Howto & Style,2009-01-18,Shaaanxo,3215340,1422,3566.0,2.278
1,UCc6W7efUSkd9YYoxOnctlFg,Entertainment,2009-06-08,Bethany Mota,10200000,483,545.0,2.087
2,UCbO9bltbkYwa56nZFQx6XJg,Howto & Style,2014-07-17,Manny Mua,4800000,426,1910.0,2.087
3,UC6jgzx2g3nlbaYkd8EMweKA,Howto & Style,2010-09-07,Jaclyn Hill,5890000,344,1348.0,2.087
4,UC9TreTE-iXwfwQl72DzDurA,Howto & Style,2009-01-04,Kandee Johnson,3900000,630,2276.0,2.166
5,UCkvK_5omS-42Ovgah8KRKtg,Howto & Style,2006-02-14,jeffreestar,16000000,357,222.0,2.087
6,UCGwPbAQdGA3_88WBuGtg9tw,Howto & Style,2010-12-03,grav3yardgirl,8560000,1536,725.0,2.087
7,UCKMugoa0uHpjUuq14yOpagw,Howto & Style,2009-06-09,Laura Lee,4400000,638,2083.0,2.120
8,UCzTKskwIc_-a0cGvCXA848Q,Howto & Style,2008-06-23,NikkieTutorials,12269235,742,312.0,2.087
9,UC9gFih9rw0zNCK3ZtoKQQyA,Comedy,2010-02-16,JennaMarbles,19900000,466,139.0,2.087


In [8]:
# Looking at the neirest neigbors of `Alex Costa` being a men fashion channel
nearest_neighbors_id = get_k_nearest_neighbors(PATH, dict_channel_ind['UCZyCposXwcyopaACep44maQ'], dict_ind_channel, k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel']).merge(channelcrawler)
nearest_neighbors

,channel,category_cc,join_date,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights
0,UCZyCposXwcyopaACep44maQ,Howto & Style,2011-09-29,Alex Costa,2120000,543,5370.0,2.4170
1,UCsTcErHg8oDvUnTzoqsYeNw,Science & Technology,2010-12-21,Unbox Therapy,15300000,1672,206.0,2.0870
2,UChNN7VBxPTiNrqjUaQd9bxA,Howto & Style,2012-09-14,Teachingmensfashion,4210000,1160,2093.0,2.1225
3,UCBJycsmduvYEL83R_U4JriQ,Science & Technology,2008-03-21,Marques Brownlee,9400000,1148,530.0,2.0870
4,UCRzzwLpLiUNIs6YOPe33eMg,Music,2012-10-22,ChainsmokersVEVO,12000000,162,384.0,2.0870
5,UC_hoQDD6zKcIqpIYLsFbBeA,People & Blogs,2013-09-22,Mo Vlogs,8280000,1500,693.0,2.0870
6,UCWFKCr40YwOZQx8FHU_ZqqQ,Howto & Style,2012-07-23,JerryRigEverything,4410000,696,1717.0,2.0870
7,UCmlsu3V3SzIm2Jmo0S0qiMg,Entertainment,2009-09-21,TechRax,6650000,501,1065.0,2.0870
8,UCMiJRAwDNSNzuYeN2uWa0pA,Science & Technology,2011-04-20,Mrwhosetheboss,2740000,1215,3367.0,2.2890
9,UCyEd6QBSgat5kkC6svyjudA,Travel & Events,2009-02-02,Mark Wiens,4900000,993,1493.0,2.0870


In [9]:
# Looking at the neirest neigbors of `Magpiepony` which is a kids film
nearest_neighbors_id = get_k_nearest_neighbors(PATH, dict_channel_ind['UCISF5OGuAtSLNF24TKTnXag'], dict_ind_channel, k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel']).merge(channelcrawler)
nearest_neighbors

,channel,category_cc,join_date,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights
0,UCISF5OGuAtSLNF24TKTnXag,Film and Animation,2013-12-17,Magpiepony,736000,304,21120.0,2.9225
1,UCfR-4yZACIMRs3wNf0hel6g,Gaming,2013-06-18,Bijuu Mike,2120000,1808,5865.0,2.4595
2,UCZBY6V8Lxmwu8gGRBOyO11w,Gaming,2014-06-16,Kubz Scouts,3160000,1440,3303.0,2.2885
3,UC_nEHeUEVNY5ZYLRWg8KoZQ,Film and Animation,2016-11-06,Wolfychu,2480000,101,4780.0,2.3640
4,UCfguohQ13Uez5UWqJbZA5Qg,Music,2012-09-09,VideoGameRapBattles,365258,93,46158.0,3.3090
5,UCHCph-_jLba_9atyCZJPLQQ,Film and Animation,2007-03-05,How It Should Have E...,9270000,355,608.0,2.0870
6,UCt4qeayu0d0Yo5NHx_3xY9A,Gaming,2012-11-12,SirSkyward,185000,1920,97587.0,3.3950
7,UCroJ5uxmGr-WOtXUPyqeh6g,Gaming,2011-06-07,Random Encounters,2780000,754,3860.0,2.2805
8,UCMsgXPD3wzzt8RxHJmXH7hQ,Entertainment,2006-02-28,Cartoon Network,5890000,5962,1095.0,2.0870
9,UC5VnC5l4dkFR0oVaPL_en2A,Gaming,2013-01-16,VenturianTale,2840000,3300,4233.0,2.3540


In [10]:
# Looking at neirest neigbors of `Beyond the trailer` a channel for analysis of films
nearest_neighbors_id = get_k_nearest_neighbors(PATH, dict_channel_ind['UCAXR2zenCwvRIyQd9ydtfaA'], dict_ind_channel, k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel']).merge(channelcrawler)
nearest_neighbors

,channel,category_cc,join_date,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights
0,UCAXR2zenCwvRIyQd9ydtfaA,Entertainment,2008-11-03,Beyond The Trailer,853000,5064,19744.0,2.8635
1,UCDiFRMQWpcp8_KD4vwIVicw,Entertainment,2012-10-12,Emergency Awesome,3210000,3949,3268.0,2.2750
2,UCa6vGFO9ty8v5KZJXQxdhaw,Entertainment,2006-09-20,Jimmy Kimmel Live,15200000,4207,212.0,2.0870
3,UCpko_-a4wgz2u_DgDgd9fqA,People & Blogs,2011-08-10,BuzzFeedVideo,19400000,6334,158.0,2.0870
4,UCBi2mrWuNuyYy4gbM6fU18Q,News & Politics,2006-08-07,ABC News,6640000,48869,978.0,2.0870
5,UCi8e0iOVk1fEOogdfu4YgfA,Film and Animation,2011-04-01,Movieclips Trailers,13900000,8084,267.0,2.0870
6,UC8-Th83bH_thdKZDJCrn88g,Comedy,2006-01-08,The Tonight Show Sta...,22100000,5450,96.0,2.0870
7,UCvC4D8onUfXzvjTOM-dBfEA,Entertainment,2005-06-16,Marvel Entertainment...,13200000,6332,302.0,2.0870
8,UC4PooiX37Pld1T8J5SYT-SQ,Entertainment,2008-09-17,Good Mythical Mornin...,15700000,2307,210.0,2.0870
9,UCCj956IF62FbT7Gouszaj9w,Entertainment,2005-11-12,BBC,7100000,20914,833.0,2.0870


### User jumper implementation

In [7]:
# We randomly choose 3 000 users and for each user we select a random pair of channel
# channels_tuple = [(channel_1, channel_2), (...)]
with open(os.path.join(COMMON_PATH, "channels_tuple_user_walk.pkl"),'rb') as f:
     channels_tuple = pickle.load(f)
f.close()

In [8]:
files = [PATH]
users_walk_tab, ranking_position_tab = get_user_walk_and_position_ratio(files, channels_tuple)

file  /dlabdata1/youtube_large/jouven/word2vec_pytorch/run_channels_more_300/channel_sampling_then_permutation/CONTEXT_True_20_SUBSAMPLING_True_0.001_LR_0.018/models/embedding0.csv.gz
n_comp  200


In [9]:
print('User walk = ' + str(users_walk_tab))
print('Ranking position ratio = ' + str(ranking_position_tab))

User walk = [0.41953733475940824]
Ranking position ratio = [0.09397382859112044]
